# 🧠 Indexing & ANN Optimization in Vector Databases

This notebook demonstrates how indexing and Approximate Nearest Neighbor (ANN) techniques using **FAISS** can significantly improve vector search performance.

We'll:
- Install necessary dependencies
- Run brute-force vs. IVF index comparisons
- Explore optimization using `nprobe`
- Mention vector compression (next steps)

In [ ]:
# ✅ Optional: Install FAISS (uncomment if not already installed)
# For CPU version:
%pip install faiss-cpu
# For GPU version (if supported):
# %pip install faiss-gpu

In [ ]:
# ✅ Check environment
import faiss
import numpy as np
from time import time

print('FAISS version:', faiss.__version__)
print('Using GPU:', faiss.get_num_gpus() > 0)

## 🔧 Step 1: Generate synthetic dataset

In [ ]:
d = 128         # Dimensions
nb = 10000      # Database size
nq = 5          # Number of query vectors

np.random.seed(42)
xb = np.random.random((nb, d)).astype('float32')
xq = np.random.random((nq, d)).astype('float32')

## 🔍 Step 2: Brute-force search using `IndexFlatL2`

In [ ]:
index_flat = faiss.IndexFlatL2(d)
index_flat.add(xb)

start = time()
D_flat, I_flat = index_flat.search(xq, k=5)
print("Brute-force search time: {:.6f}s".format(time() - start))

## ⚡ Step 3: Accelerated search using IVF (Inverted File Index)

In [ ]:
nlist = 100  # number of clusters
quantizer = faiss.IndexFlatL2(d)
index_ivf = faiss.IndexIVFFlat(quantizer, d, nlist)

index_ivf.train(xb)  # required before adding
index_ivf.add(xb)
index_ivf.nprobe = 10  # tune this to adjust accuracy-speed tradeoff

start = time()
D_ivf, I_ivf = index_ivf.search(xq, k=5)
print("IVF search time: {:.6f}s".format(time() - start))

## 📊 Summary
- `IndexFlatL2` is slow but accurate.
- `IndexIVFFlat` speeds up search using clustering.
- `nprobe` controls the tradeoff: higher = more accurate but slower.

**Next ideas:**
- Use `IndexIVFPQ` for compression
- Try `IndexHNSW` for high-speed approximate search
- Integrate this with ChromaDB or LangChain RAG pipelines
- Add result caching using Redis or in-memory hash maps